In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/brands/cetaphil/c/1938?"
params = "ptype=lst&id=1938&root=brand_menu,brand_list,Cetaphil"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&ptype=lst&id=1938&root=brand_menu,brand_list,Cetaphil&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "Cetaphil",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/brands/cetaphil/c/1938?ptype=lst&id=1938&root=brand_menu,brand_list,Cetaphil
Scraping Page 2: https://www.nykaa.com/brands/cetaphil/c/1938?page_no=2&sort=popularity&ptype=lst&id=1938&root=brand_menu,brand_list,Cetaphil&eq=desktop
Scraping Page 3: https://www.nykaa.com/brands/cetaphil/c/1938?page_no=3&sort=popularity&ptype=lst&id=1938&root=brand_menu,brand_list,Cetaphil&eq=desktop
Scraping Page 4: https://www.nykaa.com/brands/cetaphil/c/1938?page_no=4&sort=popularity&ptype=lst&id=1938&root=brand_menu,brand_list,Cetaphil&eq=desktop
Scraping Page 5: https://www.nykaa.com/brands/cetaphil/c/1938?page_no=5&sort=popularity&ptype=lst&id=1938&root=brand_menu,brand_list,Cetaphil&eq=desktop
Scraped the Total number of pages: 4


In [3]:
print(f"Products: \n {products}")

Products: 
 [{'Product Brand': 'Cetaphil', 'Product Name': 'Cetaphil Gentle Skin Cleanser Dry to Normal Skin with Niacinamide Dermatologist Recommended(125ml)', 'Product Rating': '4.5/5', 'Product Rating & Review Count': '178604ratings&12218reviews', 'Product Original Price': '₹429', 'Product Offer Price': '₹399', 'Product Discount': '7% Off', 'Product Highlight': 'BESTSELLER', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Cetaphil', 'Product Name': 'Cetaphil Brightening Night Protection Cream With Niacinamide Reduces Dark Spots- Dermatologist Tested(50gm)', 'Product Rating': '4.3/5', 'Product Rating & Review Count': '12601ratings&726reviews', 'Product Original Price': '₹1199', 'Product Offer Price': '₹1115', 'Product Discount': '7% Off', 'Product Highlight': 'BESTSELLER', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Cetaphil', 'Product Name': 'Cetaphil Moisturising Cream for dry to very dry Sensitive skin, Dermatologist Recommended(80g)', 'Product Rating': '4.5/5'

In [4]:
cetaphil  = pd.DataFrame(products)
cetaphil

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,Cetaphil,Cetaphil Gentle Skin Cleanser Dry to Normal Sk...,4.5/5,178604ratings&12218reviews,₹429,₹399,7% Off,BESTSELLER,In Stock
1,Cetaphil,Cetaphil Brightening Night Protection Cream Wi...,4.3/5,12601ratings&726reviews,₹1199,₹1115,7% Off,BESTSELLER,In Stock
2,Cetaphil,Cetaphil Moisturising Cream for dry to very dr...,4.5/5,73585ratings&4969reviews,₹669,₹615,8% Off,BESTSELLER,In Stock
3,Cetaphil,Cetaphil Oily Skin Cleanser(125ml),4.5/5,38284ratings&3261reviews,₹699,₹650,7% Off,BESTSELLER,In Stock
4,Cetaphil,Cetaphil Brightening Healthy Radiance Perfecti...,4.3/5,970ratings&32reviews,₹2299,₹1724,25% Off,None,In Stock
...,...,...,...,...,...,...,...,...,...
64,Cetaphil,Cetaphil Optimal Hydration Face & Body Care Co...,4.3/5,1709ratings&73reviews,₹2699,₹2483,8% Off,None,In Stock
65,Cetaphil,Cetaphil Sunscreen SPF 50 for Kids(150ml),4.4/5,42ratings&10reviews,₹1900,₹1748,8% Off,None,Out Of Stock
66,Cetaphil,Cetaphil Baby Advanced Protection Cream With O...,4.4/5,856ratings&29reviews,₹689,₹634,8% Off,None,Out Of Stock
67,Cetaphil,Cetaphil Optimal Hydration Body Lotion With Hy...,4.6/5,105ratings&8reviews,₹1480,₹1036,30% Off,None,Out Of Stock


In [5]:
# cetaphil.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/Cetaphil Products_Nykaa.csv", index = False)